In [144]:
import numpy as np
import math
from scipy.optimize import linprog
from abc import ABC
from collections import defaultdict
import pandas as pd

In [145]:
def generate_bit_data(n = 1000): 
    """
    Generate an array of n random bits.
    """
    return np.random.randint(0,2,n)

class Mechanism(ABC): 
    pass

class NoiseMechanism(Mechanism):
    """
    A mechanism whose queries return the true result plus noise generated by noise_generator. 
    noise_generator takes 0 arguments and returns a value of the same type as query
    """

    def __init__(self, dataset, noise_generator):
        self.dataset = dataset
        self.length = len(dataset)   
        self.noise_generator = noise_generator
        
    def get_len(self):
        return self.length
    
    def get_raw_data(self):
        return self.dataset

    def query(self, indices): 
        result = np.sum(self.dataset[indices])
        noise = self.noise_generator()
        return result + noise
    
class LaplaceMechanism(NoiseMechanism):
    """
    A noise mechanism that draws its noise from the Laplace(sensitivity/epsilon) distribution
    """
    def __init__(self, dataset, epsilon, sensitivity=1):
        self.scale = (sensitivity/epsilon)
        noise = lambda : np.random.laplace(loc=0,scale = self.scale, size=None)
        super().__init__(dataset, noise)
    
class GaussianMechanism(NoiseMechanism):
    """
    A noise mechanism that draws its noise from the Normal(2*(sensitivity^2)log(1/25/delta)/(epsilon^2))distribution
    """
    
    def __init__(self, dataset, epsilon, delta, sensitivity = 1):
        self.scale = (2 * (sensitivity ** 2) * math.log(1.25/delta)) / (epsilon ** 2)
        noise_generator = np.random.normal(loc=0, scale=self.scale, size=None)
        super().__init__(dataset, noise_generator)
    
    
def dinur_nissim(mech, epsilon = 0.5, gen_t = lambda n : n * (math.log2(n) ** 2), should_log = False):
    """
    Perform a dinur_nissim attack on the dataset, mediated by mech, a Mechanism object. 
    
    epsilon and t are the two hyperparameters of the attack. gen_t generates int-valued t as a function of n, the size of the dataset.
    
    Wrap all logging behond `should_log`
    """
    
    n = mech.get_len()
    t = int(gen_t(n))

    def generate_query():
        # Generate a random subset of [n] by generating a list of bits and convert to indices
        q_j = np.random.randint(0,2,n)
        indices2 = q_j.nonzero()
        return (q_j, indices2) 
    
    # Make t queries to the mechanism. A_ub and b_ub are respectively the coefficient matrix 
    #   and upper bound for the linear program 
    A_ub = []
    b_ub = []
    for _ in range(int(t)):
        q, indices = generate_query()
        answer = mech.query(indices)
        A_ub.append(q)
        b_ub.append(answer + epsilon)
        A_ub.append(-1 * q)
        b_ub.append(epsilon - answer)
    A_ub = np.vstack(A_ub)
    b_ub = np.array(b_ub)
    # Rounding Phase: round the results of the LP into integers
    
    if(should_log):
        print(f"Solving LP with {len(b_ub)} constraints")
    program_result = linprog(np.zeros(n), np.vstack(A_ub), b_ub, bounds=(0,1))

    if(should_log):    
        print(f"Done w/ LP")
    round_vector = np.vectorize(round, otypes=[int])
    return round_vector(program_result["x"])


def percent_reconstructed(data, results): 
    """
    What percentage of entries in data match those in results.
    """
    n = len(data)
    return 1 - ((np.count_nonzero(data!=results))/n)

In [146]:
# Very simple example
# data = generate_bit_data(n=500)
# mech = LaplaceMechanism(data, epsilon=100)
# dinur_nissim(mech, gen_t=lambda n : n)

In [147]:
def run_experiment(ns, mechanism_class, mech_args, dn_epsilons, t_generators): 
    
    results_df =  pd.DataFrame(columns=["n", 
                              *list(mech_args[0].keys()), 
                              "dn_espsilon", 
                              "num_queries", 
                              "percent_reconstructed"]) 
    
    def make_entry(mech_arg, n, dn_epsilon, t, pc_reconstructed): 
        res = {x : [mech_arg[x]] for x in mech_arg.keys()}
        res.update (
            {
                "n" : [n],
                "dn_espsilon" : [dn_epsilon],
                "num_queries" : [t],
                "percent_reconstructed" : pc_reconstructed
            }
        )
        return pd.DataFrame.from_dict(res)
 
    for n in ns:
        for mech_arg in mech_args:
            # New dataset and mech for each size/mechanism params pair
            data = generate_bit_data(n=n)
            mech = mechanism_class(data, **mech_arg)
            for dn_epsilon in dn_epsilons: 
                for t_generator in t_generators:
                    result = dinur_nissim(mech, dn_epsilon,t_generator)
                    # Lambdas are hard to log so we log its application. We can probably abstract this better.
                    t = int(t_generator(n))
                    
                    # Append new results 
                    entry = make_entry(mech_arg, n, dn_epsilon, t, percent_reconstructed(data, result))
                    results_df = pd.concat([results_df, entry], ignore_index=True)           
    
    return results_df

In [ ]:
# Example usage. Essentially runs over the cartesian product of the lists [ns, mech_args, dn_epsilons, t_generators]
r = run_experiment(
    ns = [500+100*x for x in range(2)], 
    mechanism_class = LaplaceMechanism, 
    mech_args = [{"epsilon": x} for x in [.3,.3,.4,.5,.6, .8, 1, 1.2]], 
    dn_epsilons=[.5], 
    t_generators=[lambda n : n])
r